In [1]:
# What does lru_cache do?
from functools import lru_cache

@lru_cache(1)
def addxy(x, y):
    print('add {} and {}'.format(x, y))
    return x+y

print(addxy(1, 2))
print(addxy(1, 2))
print(addxy(3, 4))

add 1 and 2
3
3
add 3 and 4
7


In [2]:
import glob
from collections import namedtuple
import os
import csv
import SimpleITK as sitk
import numpy as np
import torch as t
from torch.utils.data import Dataset, DataLoader

In [3]:
CandidateInfoTuple = namedtuple(
'CandidateInfoTuple', # this namedtuple's name
'isNodule_bool, diameter_mm, series_uid, center_xyz')

In [4]:
# get the name of .mhd file I downloaded and store these information in cache
# no return, just store
requireOnDisk_bool = True
@lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool=requireOnDisk_bool):
    # a default parameter of this function
    mhd_list = glob.glob('./luna16/data/subset*/*.mhd')
    # mhd_list is a list of file paths, like '/subset7/534991.mhd'
    # os.path.split(p) returns a list of os.path and p, [path, p]
    # -1 we get p, and :-4 we drop .mhd and get only the name
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}
    return(presentOnDisk_set)

presentOnDisk_set = getCandidateInfoList(requireOnDisk_bool)

In [5]:
# get information about diameter for each id
diameter_dict = {}
with open('./luna16/data/annotations.csv') as f:
    # csv.reader: read f in csv version, so row is a list of each element
    # 1: because the first row is header
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
        annotationCenter_xyz = tuple(float(x) for x in row[1:4])
        annotationDiameter_mm = float(row[4])
        # dict.setdefault(a, b): if a is in dict, then return dict[a],
        # if a is not in dict, then set dict[a]=b and return
        diameter_dict.setdefault(series_uid, []).append(
            (annotationCenter_xyz, annotationDiameter_mm)
        )

In [6]:
# get information of each nodule candidate from candidate file
## use distance rather than abs
def euclidean_distance(p1, p2):
    distance = 0
    for i in range(len(p1)):
        distance += (p1[i]-p2[i])**2
    return distance**0.5

candidateInfo_list = []
with open('./luna16/data/candidates.csv') as f:
    for row in list(csv.reader(f))[1:]:
        series_uid = row[0]
        # check whether id is in the cache (in our subset file)
        if series_uid not in presentOnDisk_set and requireOnDisk_bool:
            continue # skip this one
        # is nodule or not
        isNodule_bool = bool(int(row[4]))
        candidateCenter_xyz = tuple([float(x) for x in row[1:4]])
        # default diameter is 0
        candidateDiameter_mm = 0.0
        # dict.get(a, b): if a is in dict, then return dict[a],
        # if a is not in dict, then return b
        for annotation_tup in diameter_dict.get(series_uid, []):
            annotationCenter_xyz, annotationDiameter_mm = annotation_tup
            # distance measures how annotationcenter and candidatecenter apart
            # If find one, then get annotationdiameter
            # If not, then get the next annotation point in this same id
            # If couldn't find in the end, then the diameter is 0
            distance = euclidean_distance(annotationCenter_xyz, candidateCenter_xyz)
            # my method
            if distance > annotationDiameter_mm / 2:
                continue
            else:
                candidateDiameter_mm = annotationDiameter_mm
                break
            # method in book
#             for i in range(3):
#                 delta_mm = abs(candidateCenter_xyz[i]-annotationCenter_xyz[i])
#                 if delta_mm > annotationDiameter_mm / 4:
#                     break
#             else:
#                 candidateDiameter_mm = annotationDiameter_mm
#                 break
            
        # form the list of candidate info
        candidateInfo_list.append(
            CandidateInfoTuple(isNodule_bool,
                               candidateDiameter_mm, 
                               series_uid, 
                               candidateCenter_xyz)
        )
                    

In [7]:
# all the data with large diameter are at front
candidateInfo_list.sort(reverse=True)

In [8]:
# get imgdata
def getct(series_uid):
    mhd_path = glob.glob('./luna16/data/subset*/{}.mhd'.format(series_uid))[0]
    # ct_mhd contains the info of dimension, convert matrix etc.
    ct_mhd = sitk.ReadImage(mhd_path)
    ct_np = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)
    # set the min and max value based on HU units
    # clip: set the value lower or higher than the thresholds to the thresholds
    ct_np.clip( -1000, 1000, ct_np)
    return (ct_mhd, ct_np)

In [9]:
ct_mhd, ct_np = getct('1.3.6.1.4.1.14519.5.2.1.6279.6001.487268565754493433372433148666')
print(ct_mhd)

Image (0x7ff322124830)
  RTTI typeinfo:   itk::Image<short, 3u>
  Reference Count: 1
  Modified Time: 897
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 870
  UpdateMTime: 896
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 133]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 133]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [512, 512, 133]
  Spacing: [0.78125, 0.78125, 2.5]
  Origin: [-191.2, -185.5, -359]
  Direction: 
1 0 0
0 1 0
0 0 1

  IndexToPointMatrix: 
0.78125 0 0
0 0.78125 0
0 0 2.5

  PointToIndexMatrix: 
1.28 0 0
0 1.28 0
0 0 0.4

  Inverse Direction: 
1 0 0
0 1 0
0 0 1

  PixelContainer: 
    ImportImageContainer (0x7ff322137080)
      RTTI typeinfo:   itk::ImportImageContainer<unsigned long, short>
      Reference Count

In [10]:
ct_np.shape

(133, 512, 512)

In [11]:
IrcTuple = namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = namedtuple('XyzTuple', ['x', 'y', 'z'])

def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1]
    origin_xyz = np.array(origin_xyz)
    vxSize_xyz = np.array(vxSize_xyz)
    # * is the multiplication of each value, @ is inner multiplication
    coord_xyz = direction_a @ (cri_a * vxSize_xyz) + origin_xyz
    # *coord_xyz: input all the parameters as tuple
    # **coord_xyz: input all the parameters as dic
    return XyzTuple(*coord_xyz)

def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    coord_xyz = np.array(coord_xyz)
    origin_xyz = np.array(origin_xyz)
    vxSize_xyz = np.array(vxSize_xyz)
    cri_a = (coord_xyz - origin_xyz) @ np.linalg.inv(direction_a) / vxSize_xyz
    cri_a = np.round(cri_a)
    # int doesn't do half adjusting
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

In [12]:
def getRawCandidate(center_xyz, origin_xyz, vxSize_xyz, 
                    direction_a, width_irc, ct_np):
    # first get (index, row, channel)
    # the format of center_irc is: IrcTuple(index=80, row=254, col=400)
    center_irc = xyz2irc(center_xyz, origin_xyz, vxSize_xyz, direction_a)
    # get the index of centered candidate
    slice_list = []
    for axis, center_val in enumerate(center_irc):
        start_ndx = int(round(center_val-width_irc[axis]/2))
        end_ndx = int(start_ndx+width_irc[axis])
        slice_list.append(slice(start_ndx, end_ndx))
    # slice at each dimension to form centered candidate
    ct_chunk = ct_np[tuple(slice_list)]
    return (ct_chunk, center_irc)

In [13]:
def getitem(ndx):
    candidateInfo_tup = candidateInfo_list[ndx]
    width_irc = (32, 48, 48)
    ct_mhd, ct_np = getct(candidateInfo_tup.series_uid)
    origin_xyz = ct_mhd.GetOrigin()
    vxSize_xyz = ct_mhd.GetSpacing()
    direction_a = np.array(ct_mhd.GetDirection()).reshape(3,3)
    candidate_a, center_irc = getRawCandidate(candidateInfo_tup.center_xyz,
                                             origin_xyz,
                                             vxSize_xyz,
                                             direction_a,
                                             width_irc, 
                                             ct_np)
    # transform to tensor
    candidate_t = t.from_numpy(candidate_a)
    candidate_t = candidate_t.to(t.float32)
    # add a new dimension called 'channel'
    candidate_t = candidate_t.unsqueeze(0)
    # now we should get labels
    pos_t = t.tensor([not candidateInfo_tup.isNodule_bool,
                     candidateInfo_tup.isNodule_bool],
                     # this t.long change true to 1, false to 0
                     dtype=t.long)
    return (candidate_t, pos_t, 
            candidateInfo_tup.series_uid, 
            candidateInfo_tup.center_xyz)

In [14]:
# we want a 8:2 split
val_stride = 5

In [15]:
def get_train_val(val_stride = 0,
                 isValSet_bool = None,
                 series_uid = None):
    # create a copy, so that changing value in candidateInfo_part wouldn't 
    # change the value in candidateInfo_list
    candidateInfo_list_part = candidateInfo_list.copy()
    # if we want only part of the data with series_uid
    if series_uid:
        candidateInfo_list_part = [
            x for x in candidateInfo_list_part if x.series_uid == series_uid
        ]
    # if we want to get validation set
    if isValSet_bool:
        # assert means if val_stride<=0, then assert an error with val_stride
        assert val_stride>0, val_stride
        candidateInfo_list_part = candidateInfo_list_part[::val_stride]
        assert candidateInfo_list_part
    elif val_stride>0:
        # delete the validation data get the training data
        del candidateInfo_list_part[::val_stride]
        assert candidateInfo_list_part
    return candidateInfo_list_part

In [16]:
trainSet = get_train_val(val_stride, isValSet_bool=False)
valSet = get_train_val(val_stride, isValSet_bool=True)
print(len(trainSet), len(valSet))

178530 44633


In [17]:
print(getitem(1))

(tensor([[[[  41.,   65.,   69.,  ..., -896., -900., -898.],
          [  67.,   70.,   62.,  ..., -889., -899., -916.],
          [  65.,   86.,   88.,  ..., -865., -875., -903.],
          ...,
          [-803., -812., -839.,  ..., -843., -847., -839.],
          [-840., -842., -848.,  ..., -824., -830., -817.],
          [-831., -839., -849.,  ..., -818., -817., -809.]],

         [[  48.,   42.,   22.,  ..., -880., -879., -874.],
          [  61.,   47.,   12.,  ..., -899., -885., -882.],
          [  42.,   39.,   -1.,  ..., -909., -909., -887.],
          ...,
          [-853., -867., -872.,  ..., -859., -855., -886.],
          [-879., -890., -871.,  ..., -867., -862., -880.],
          [-880., -874., -842.,  ..., -882., -866., -860.]],

         [[ -30., -109., -195.,  ..., -874., -883., -871.],
          [ -97., -165., -232.,  ..., -892., -895., -882.],
          [-117., -196., -259.,  ..., -890., -902., -893.],
          ...,
          [-874., -884., -913.,  ..., -892., -847.

In [18]:
################################
# subclass Dataset
class LunaDataset(Dataset):
    def __init__(self, val_stride=0, isValSet_bool=None, series_uid=None):
        self.candidateInfo_list = candidateInfo_list.copy()
        if series_uid:
            self.candidateInfo_list = [
                x for x in self.candidateInfo_list if x.series_uid==series_uid
            ]
        if isValSet_bool:
            assert val_stride>0, val_stride
            self.candidateInfo_list = self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
        elif val_stride>0:
            # delete the validation data get the training data
            del self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
            
    # the function we need to define
    def __len__(self):
        return len(self.candidateInfo_list)
    
    # the function we need to define
    def __getitem__(self, ndx):
        candidateInfo_tup = self.candidateInfo_list[ndx]
        width_irc = (32, 48, 48)
        ct_mhd, ct_np = getct(candidateInfo_tup.series_uid)
        origin_xyz = ct_mhd.GetOrigin()
        vxSize_xyz = ct_mhd.GetSpacing()
        direction_a = np.array(ct_mhd.GetDirection()).reshape(3,3)
        candidate_a, center_irc = getRawCandidate(candidateInfo_tup.center_xyz,
                                                 origin_xyz,
                                                 vxSize_xyz,
                                                 direction_a,
                                                 width_irc, 
                                                 ct_np)
        # transform to tensor
        candidate_t = t.from_numpy(candidate_a)
        candidate_t = candidate_t.to(t.float32)
        # add a new dimension called 'channel'
        candidate_t = candidate_t.unsqueeze(0)
        # now we should get labels
        pos_t = t.tensor([not candidateInfo_tup.isNodule_bool,
                         candidateInfo_tup.isNodule_bool],
                         # this t.long change true to 1, false to 0
                         dtype=t.long)
        return (candidate_t, pos_t, 
                candidateInfo_tup.series_uid, 
                candidateInfo_tup.center_xyz)

In [19]:
# we get the same result
LunaDataset()[1]

(tensor([[[[  41.,   65.,   69.,  ..., -896., -900., -898.],
           [  67.,   70.,   62.,  ..., -889., -899., -916.],
           [  65.,   86.,   88.,  ..., -865., -875., -903.],
           ...,
           [-803., -812., -839.,  ..., -843., -847., -839.],
           [-840., -842., -848.,  ..., -824., -830., -817.],
           [-831., -839., -849.,  ..., -818., -817., -809.]],
 
          [[  48.,   42.,   22.,  ..., -880., -879., -874.],
           [  61.,   47.,   12.,  ..., -899., -885., -882.],
           [  42.,   39.,   -1.,  ..., -909., -909., -887.],
           ...,
           [-853., -867., -872.,  ..., -859., -855., -886.],
           [-879., -890., -871.,  ..., -867., -862., -880.],
           [-880., -874., -842.,  ..., -882., -866., -860.]],
 
          [[ -30., -109., -195.,  ..., -874., -883., -871.],
           [ -97., -165., -232.,  ..., -892., -895., -882.],
           [-117., -196., -259.,  ..., -890., -902., -893.],
           ...,
           [-874., -884., -913.

In [20]:
# store it in DataLoader
trainSet = LunaDataset(val_stride, isValSet_bool=False)
batch_size = 16
trainLoader = DataLoader(trainSet, batch_size=batch_size)

In [ ]:
for data, label, idd, xyz in trainLoader:
    if idd == '1.3.6.1.4.1.14519.5.2.1.6279.6001.487268565754493433372433148666':
        print(label)
        break

In [21]:
trainLoader.dataset[0]

(tensor([[[[  41.,   65.,   69.,  ..., -896., -900., -898.],
           [  67.,   70.,   62.,  ..., -889., -899., -916.],
           [  65.,   86.,   88.,  ..., -865., -875., -903.],
           ...,
           [-803., -812., -839.,  ..., -843., -847., -839.],
           [-840., -842., -848.,  ..., -824., -830., -817.],
           [-831., -839., -849.,  ..., -818., -817., -809.]],
 
          [[  48.,   42.,   22.,  ..., -880., -879., -874.],
           [  61.,   47.,   12.,  ..., -899., -885., -882.],
           [  42.,   39.,   -1.,  ..., -909., -909., -887.],
           ...,
           [-853., -867., -872.,  ..., -859., -855., -886.],
           [-879., -890., -871.,  ..., -867., -862., -880.],
           [-880., -874., -842.,  ..., -882., -866., -860.]],
 
          [[ -30., -109., -195.,  ..., -874., -883., -871.],
           [ -97., -165., -232.,  ..., -892., -895., -882.],
           [-117., -196., -259.,  ..., -890., -902., -893.],
           ...,
           [-874., -884., -913.